# NEURAL NETWORKS - COMP258
## MIDTERM EXAM - EXERCISE 1
## Nestor Romero - 301133331

In [1]:
# Basic libraries imports
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
######## 1. LOAD DATASET FILE INTO DATA FRAME

bank_notes_df = pd.read_csv('banknote_authentication.csv', names=['variance','skewness','curtosis','entropy', 'class'])
print('GENERAL DATASET INFORMATION\n', bank_notes_df.describe(),'\n')
print('DATA SAMPLE\n', bank_notes_df.head(), '\n')
print('CLASS VALUE COUNTS\n', bank_notes_df['class'].value_counts(), '\n')

GENERAL DATASET INFORMATION
           variance     skewness     curtosis      entropy        class
count  1372.000000  1372.000000  1372.000000  1372.000000  1372.000000
mean      0.433735     1.922353     1.397627    -1.191657     0.444606
std       2.842763     5.869047     4.310030     2.101013     0.497103
min      -7.042100   -13.773100    -5.286100    -8.548200     0.000000
25%      -1.773000    -1.708200    -1.574975    -2.413450     0.000000
50%       0.496180     2.319650     0.616630    -0.586650     0.000000
75%       2.821475     6.814625     3.179250     0.394810     1.000000
max       6.824800    12.951600    17.927400     2.449500     1.000000 

DATA SAMPLE
    variance  skewness  curtosis  entropy  class
0   3.62160    8.6661   -2.8073 -0.44699      0
1   4.54590    8.1674   -2.4586 -1.46210      0
2   3.86600   -2.6383    1.9242  0.10645      0
3   3.45660    9.5228   -4.0112 -3.59440      0
4   0.32924   -4.4552    4.5718 -0.98880      0 

CLASS VALUE COUNTS
 0    76

In [3]:
#function to create random weights to initialize a layer
def create_weights(num_units, num_inputs):
    '''
    Create a matrix of random values with shape (num_units, num_inputs)
    num_units = Number of neurons in layer
    num_inputs = Number of inputs from previous layer
    '''
    weights = 2* np.array(np.random.rand(num_units, num_inputs)) - 1
    return weights

# sigmoid function
sigmoid = lambda x: 1/(1 + np.exp(-x));

In [4]:
# function to execute final feed forward step with the trained network
def feed_forward(whl, wol, inputs, outputs):
    '''
    This function calculates only the feed forward step for the network
    It is used to calculate the final results of the network
    whl = weights hidden layer
    wol = weights output layer
    '''
    num_records = inputs.shape[0]
    results = a = np.zeros(outputs.shape)
    
    for i in range(0, num_records):
        
        #for each row of values and results
        x = inputs[i, :].T;    # current set of values
        d = outputs[i];        # current set of results
        
        # FEEDFORWARD SECTION
        # HIDDEN LAYER
        # Calculate weighted sum and output
        v1 = np.dot(whl,x);
        y1 = sigmoid(v1);
        
        # OUTPUT LAYER
        # Calculate weighted sum and output
        v = np.dot(wol,y1);
        y = sigmoid(v);
        
        #create results
#         print(f'd = {d} y = {y}')
        results[i] = y
        
    return results

In [5]:
# function to execute backpropagation algorithm
def back_propagation(whl, wol, inputs, outputs):
    '''
    This function calculates the feed forward and back propagation steps for the network
    whl = weights hidden layer
    wol = weights output layer
    '''
    # learning rate
    alpha = 0.9;
    
    ### size variables to avoid hardcoding
    num_records = inputs.shape[0]
    size_input = inputs.shape[1]
    size_output = wol.shape[0]
    units_hidden = whl.shape[0]
    units_output = wol.shape[0]
    ###
    
    for i in range(0,num_records):
        
#         print(f'\n\nCURRENT ROW: {i}')
        
        #for each row of values and results
        x = inputs[i, :].T;    # current set of values
        d = outputs[i];        # current set of results
        
        # FEEDFORWARD SECTION
        # HIDDEN LAYER
        # Calculate weighted sum and output
        v1 = np.dot(whl,x);
        y1 = sigmoid(v1);

        
        # OUTPUT LAYER
        # Calculate weighted sum and output
        v = np.dot(wol,y1);
        y = sigmoid(v);
        
        # CALCULATE ERROR AND DELTA
        error = d - y;
        delta = y * (1 - y) * error; # sigmoid prime!!!
        
        #
        # BACKPROPAGATION SECTION
        e1 = np.dot(wol.T, delta);
        delta1 = y1 * (1 - y1) * e1;  
        
        # Adjust the weights according to the learning rule
        # HIDDEN LAYER
        delta1.shape=(units_hidden,1)   # column vector of deltas for the hidden layer
        x.shape=(1,size_input)          # row vector of the current input
        
        dwhl = alpha * np.dot(delta1,x);
        whl = whl + dwhl;
        
        # OUTPUT LAYER
        delta.shape = (size_output,1)
        y1.shape = (1, units_hidden)
        
        dwol = alpha*np.dot(delta,y1);
        wol = wol + dwol;
        
    return whl, wol;

In [6]:
######## 2. EXTRACT INPUT MATRIX AND OUTPUT COLUMN

X = bank_notes_df.iloc[:,0:4].to_numpy() #Input values (n,4)
y = bank_notes_df['class'].to_numpy()   #Output values (n)

######## 3. SPLIT DATASET INTO TRAIN AND TEST
# (70%-30% split) random_state used for replicable results

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=301133331)

X_train

array([[  2.8237  ,   2.8597  ,   0.19678 ,   0.57196 ],
       [  5.681   ,   7.795   ,  -2.6848  ,  -0.92544 ],
       [ -0.4294  ,  -0.14693 ,   0.044265,  -0.15605 ],
       ...,
       [ -1.7263  ,  -6.0237  ,   5.2419  ,   0.29524 ],
       [ -4.244   , -13.0634  ,  17.1116  ,  -2.8017  ],
       [ -1.5768  ,  10.843   ,   2.5462  ,  -2.9362  ]])

In [7]:
# MAIN PROGRAM EXECUTION

######## 4. INITIALIZE WEIGHTS

# Create initial random weights for all layers
W1 = create_weights(6,4) # Hidden layer 6 units and 4 inputs
W2 = create_weights(1,6) # Output layer 1 unit 6 inputs

print('###\tINITIALIZE NETWORK WEIGHTS')
print('\tHidden Layer Weights - W1')
print(W1)
print()
print('\tOutput Layer Weights - W2')
print(W2)

print('\n###\tNETWORK ARCHITECTURE')
print(f'Weights Hidden Layer Shape (W1) {W1.shape}')
print(f'Weights Output Layer Shape (W2) {W2.shape}')

print()
print('###\tTRAINING NETWORK - BACK-PROPAGATION')
print(f'Train Inputs Shape {X_train.shape}')
print(f'Train Outputs Shape {y_train.shape}')
# Use back propagation to obtain NN weights
W1, W2 = back_propagation(W1, W2, X_train, y_train)

###	INITIALIZE NETWORK WEIGHTS
	Hidden Layer Weights - W1
[[ 0.18332064 -0.75068694 -0.95455357  0.94320008]
 [ 0.24668493  0.37144921 -0.72560339 -0.90532626]
 [ 0.40960049  0.58089953  0.59263356  0.26810477]
 [-0.94909102  0.45420361 -0.61628618 -0.64252646]
 [-0.55559508 -0.68170695  0.51295861 -0.55816557]
 [ 0.94047852  0.13453754 -0.60079859  0.01735042]]

	Output Layer Weights - W2
[[ 0.17056033  0.19754935 -0.57823532  0.38417356  0.14968244 -0.89191795]]

###	NETWORK ARCHITECTURE
Weights Hidden Layer Shape (W1) (6, 4)
Weights Output Layer Shape (W2) (1, 6)

###	TRAINING NETWORK - BACK-PROPAGATION
Train Inputs Shape (960, 4)
Train Outputs Shape (960,)


In [8]:
print('\n###\tTESTING DATA')
print(f'Test Inputs Shape {X_test.shape}')
print(f'Test Outputs Shape {y_test.shape}')

######## 5. CALCULATE OUTPUT OF THE NETWORK WITH A SAMPLE

# Calculate results for test values with the weights obtained from back_propagation
y_predicted = feed_forward(W1, W2, X_test, y_test)

# transform result into an integer for comparison and accuracy calculation
y_output = [ 1 if y > 0.5 else 0 for y in y_predicted ]

sample_size = 10
print(f'\n###\tFORMATTED SAMPLE RESULTS ({sample_size})')

# Print 5 samples of data results vs predicted
for i in range(0, sample_size):
    print(f'x = {X_test[i]},\ty = {y_test[i]},\tpredicted_value = {y_predicted[i]},\ty_output = {y_output[i]}\n')


accuracy = accuracy_score(y_test,y_output)
print(f'\n### ACCURACY SCORE: {accuracy}')


###	TESTING DATA
Test Inputs Shape (412, 4)
Test Outputs Shape (412,)

###	FORMATTED SAMPLE RESULTS (10)
x = [ 0.7057 -5.4981  8.3368 -2.8715],	y = 0,	predicted_value = 0.06405193807190121,	y_output = 0

x = [ 0.33111  4.5731   2.057   -0.18967],	y = 0,	predicted_value = 0.025550927161098053,	y_output = 0

x = [-0.27802  8.1881  -3.1338  -2.5276 ],	y = 0,	predicted_value = 0.06297311215956405,	y_output = 0

x = [-2.2214  -0.23798  0.56008  0.05602],	y = 1,	predicted_value = 0.9969569112680016,	y_output = 1

x = [ 1.9476 -4.7738  8.527  -1.8668],	y = 0,	predicted_value = 0.02238962382584315,	y_output = 0

x = [ 5.6084 10.3009 -4.8003 -4.3534],	y = 0,	predicted_value = 0.0367653800055792,	y_output = 0

x = [-2.121   -0.05588  1.949    1.353  ],	y = 1,	predicted_value = 0.9799807590065568,	y_output = 1

x = [-3.2778  1.8023  0.1805 -2.3931],	y = 1,	predicted_value = 0.9948376163715977,	y_output = 1

x = [ 0.89512  4.7738  -4.8431  -5.5909 ],	y = 1,	predicted_value = 0.6077699601056378,	y